<a href="https://colab.research.google.com/github/PedroAVelezP/EduCheck-AI/blob/Deepseek_R1/Notebooks/Debug_EduCheck_Deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [1]:
!git clone https://github.com/PedroAVelezP/EduCheck-AI.git

Cloning into 'EduCheck-AI'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 118 (delta 50), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 71.39 KiB | 5.95 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
!pip install gradio
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [3]:
from IPython.display import clear_output
import numpy as np
import os
import threading
import subprocess
import requests
import json

import time
import pandas as pd
import gradio as gr

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

In [ ]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
ollama()
!ollama pull llama3.2
clear_output()

!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

# Ejecución

In [ ]:
%cd /content/EduCheck-AI
!python main.py --preset debug

# Debug

In [ ]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
ollama()
!ollama pull deepseek-r1:8b
clear_output()

!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

In [31]:
class SimpleQA(Component):
    def __init__(self, model_client: OllamaClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

qa_template = """<SYS>
You are an AI specialized in scoring the closeness between the correct answer (CA) and the given answer (GA), for this you will have to analyze the question that was being asked before getting the given answer.

The output format should be as follows:
###Score: (0-100)
###Justification:

Consider the initial question to give your final score, since being specialized topics it may be the case that the answer given (GA) is completely correct despite being different from the answer marked as correct (CA), at the same time consider the degree of knowledge that the person seems to have of the topic, since it may be the case where the person tries to guess the correct answer.
The question asked was: ¿Qué hormona regula los niveles de glucosa en sangre y dónde se produce?
The correct answer should be: La insulina, producida por las células beta del páncreas.
The answer given was: insulina

</SYS>
"""

# Crear cliente y llamada a SimpleQA
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "deepseek-r1:8b"}
}
qa = SimpleQA(**model)
Entrada = qa(f"")
respuesta_generada = Entrada.data

print(respuesta_generada)

<think>
Okay, so I need to figure out how to score the closeness between the correct answer and the given answer for this question. The question is about which hormone regulates blood glucose levels and where it's produced.

First, let me look at the correct answer. It says that insulin is produced by beta cells in the pancreas. That makes sense because I remember from biology class that insulin is released by the pancreas to help regulate blood sugar.

Now, the given answer is just "insulina," which is Spanish for insulin. So the GA (given answer) correctly identifies the hormone as insulin without specifying its source or function. But the correct answer not only names the hormone but also mentions where it's produced.

I should consider how similar these two answers are. The GA is missing some details about the production location, but it does get the name right. Maybe the person knew the answer but didn't include all the required information. Alternatively, they might have just gue